<img src='./img/LogoWekeo_Copernicus_RGB_0.png' alt='' align='centre' width='30%'></img>

# WEkEO Harmonized Data Access (HDA) API - How-To

[WEkEO](https://www.wekeo.eu/) is the EU Copernicus DIAS (Data and Information Access Service) reference service for environmental data, virtual processing environments and skilled user support.

WEkEO offers access to a variety of data, including different parameters sensored from Sentinel-1, Sentinel-2 and Sentinel-3. It further offers access to climate reanalysis and seasonal forecast data.

The [Harmonised Data Access (HDA) API](https://www.wekeo.eu/docs/harmonised-data-access-api), a REST interface, allows users to subset and download datasets from WEkEO.

This [notebook](hhttps://www.wekeo.eu/docs/using-jupyter) is a step-by-step guide on how to search for and download data from WEkEO using the `HDA API`.

Follow the following steps:

 - [1. Install the WEkEO HDA client](#wekeo_hda_install)
 - [2. Search for datasets on WEkEO](#wekeo_search)
 - [3. Get the Dataset ID](#wekeo_dataset_id)
 - [4. Configure the WEkEO API Authentication](#wekeo_hda_auth)
 - [5. Load data descriptor file and request data](#wekeo_json)
 - [6. Download requested data](#wekeo_download)

#### Load required libraries

In [ ]:
import os
import sys
import json
import time
import base64
from IPython.core.display import HTML

import requests
import warnings
warnings.filterwarnings('ignore')

### <a id='wekeo_hda_install'></a>1. Install the WEkEO HDA client

The WEkEO HDA client is a python based library. It provides support for both Python 2.7.x and Python 3.

In order to install the WEkEO HDA client via the package management system pip, you have to running on Unix/Linux the command shown below.

In [ ]:
pip install -U hda

Please verify the following requirements are installed before skipping to the next step:
   - Python 3
   - requests
   - tqdm

#### Load WEkEO HDA client

The hda client provides a fully compliant Python 3 client that can be used to search and download products using the Harmonized Data Access WEkEO API.
HDA is RESTful interface allowing users to search and download WEkEO datasets.
Documentation about its usage can be found at https://www.wekeo.eu/.

In [ ]:
from hda import Client

<hr>

### <a id='wekeo_search'></a>2. Search for datasets on WEkEO

Under [WEkEO DATA](https://wekeo.eu/data?view=catalogue). Clicking the + to add a layer, opens a catalogue search where you cna use free text.You can look for example for *`sentinel-3`* and browse through the results. You can also use the filter options on the left to refine your search and look by satellite plaform, sensor, Copernicus service, area (region of interest), general time period (past or future), as well as through a variety of flags.


You can click on the dataset you are interested in and you will be guided to a range of details including the dataset temporal and spatial extent, collection ID, and metadata.

<br>

<div style='text-align:center;'>
<figure><img src='./img/wekeo_data_search2.png' width='70%' />
    <figcaption><i>WEkEO interface to search for datasets</i></figcaption>
</figure>
</div>

### <a id='wekeo_dataset_id'></a>3. Get the Dataset ID 

The dataset description provides the following information:
- **Abstract**, containing a general description of the dataset,
- **Classification**, including the Dataset ID 
- **Resources**, such as a link to the Product Data Format Specification guide, and JSON metadata
- **Contacts**, where you can find further information about the data source from its provider.  

You need the `Dataset ID` to request data from the Harmonised Data Access API. 

For `OLCI Level 1B Full Resolution - Sentinel-3` data for example, the collection ID is `EO:EUM:DAT:SENTINEL-3:OL_1_EFR___`.

<br>

<div style='text-align:center;'>
<figure><img src='./img/wekeo_collection_id2.png' width='60%' />
    <figcaption><i>Dataset information on WEkEO</i></figcaption>
</figure>
</div>
<br>

Let's store the Dataset ID as a variable called `dataset_id` to be used later.

In [ ]:
dataset_id = "EO:EUM:DAT:SENTINEL-3:OL_1_EFR___"

### <a id='wekeo_hda_auth'></a>4. Configure the WEkEO API Authentication

In order to interact with WEkEO's Harmonised Data Access API, each user first makes sure the file "$HOME/.hdarc" exists with the URL to the API end point and your user and password.

For example, to search for the file .hdarc in the $HOME diretory, the user would open a terminale and run the following command:

Then he could copy the code below in the file "$HOME/.hdarc" (in your Unix/Linux environment) and adapt the following template with the credentials of your WEkEO account:

If he doesn't have a WEkEO account, please self register at the WEkEO registration page https://my.wekeo.eu/web/guest/user-registration.

### <a id='wekeo_json'></a>5. Load data descriptor file and request data

The Harmonised Data Access API can read your data request from a `JSON` file. In this JSON-based file, you can describe the dataset you are interested in downloading. The file is in principle a dictionary. Following keys can be defined:
- `datasetID`: the dataset's collection ID
- `stringChoiceValues`: type of dataset, e.g. 'Non Time Critical'
- `dataRangeSelectValues`: time period you would like to retrieve data
- `boundingBoxValues`: optional to define a subset of a global field

See an example of a `data descriptor` file [here](./olci_data_descriptor.json). You can also get a specific example of a `JSON` file for a particular query from the [WEkEO DATA](https://wekeo.eu/data?view=catalogue) portal when you search as above - you just need to click on API request and the information needed for the `JSON` file will be displayed.

<br>

<div style='text-align:center;'>
<figure><img src='./img/Mindule_eg_viewer3.PNG' width='100%' />
    <figcaption><i>Example of search definition in the WEkEO portal, clicking on "API Request" will provide the data request in JSON format (see example below)</i></figcaption>
</figure>
</div>
<br>

<br>

<div style='text-align:center;'>
<figure><img src='./img/Mindule_API_request.PNG' width='60%' />
    <figcaption><i>Displaying a JSON query from a request made to the Harmonised Data Access API through the data portal</i></figcaption>
</figure>
</div>
<br>

You can load the `JSON` file with `json.load()`. Alternatively, you can copy paste the dictionary describing your data into a cell, as done below. Make sure to choose one option if you are inputting your own example! 

In [ ]:
try:
    with open('./olci_data_descriptor.json', 'r') as f:
        data = json.load(f)
    print('Your JSON file:')
    print(data)
except:
    print('Your JSON file is not in the correct format, or is not found, please check it!')

In [ ]:
data = {
  "datasetId": "EO:EUM:DAT:SENTINEL-3:OL_1_EFR___",
  "boundingBoxValues": [
    {
      "name": "bbox",
      "bbox": [
        134.2477516188146,
        22.00625971252353,
        137.58597074610276,
        25.18812983958223
      ]
    }
  ],
  "dateRangeSelectValues": [
    {
      "name": "position",
      "start": "2021-09-29T00:00:00.000Z",
      "end": "2021-09-30T00:00:00.000Z"
    }
  ],
  "stringChoiceValues": [
    {
      "name": "platformname",
      "value": "Sentinel-3"
    },
    {
      "name": "producttype",
      "value": "OL_1_EFR___"
    },
    {
      "name": "timeliness",
      "value": "Non Time Critical"
    }
  ]
}

### <a id='wekeo_download'></a>6. Download requested data

As a final step, you can use the WEkEO HDA API to request data from the datasets listed in the WEkEO catalogue and to download it. 

In [ ]:
c = Client(debug=True)

matches = c.search(data)
print(matches)
matches.download()

<hr>